In [1]:
import pandas as pd
import numpy as np
from time import time
from config_loader import load
import argparse
import sys
import seaborn as sns
from MyDataUnderstanding import featureAnalysis
from MyPreprocessing import MyPreprocessing
import numpy as np
from time import time
import matplotlib.pyplot as plt
from model.models import models_perform
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from statistics import mean
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from detect_outliers import detect_outliers
from MyFeatureSelection import MyFeatureSelection

In [2]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import SVC
from xgboost import XGBClassifier as XGB
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from model.MyIBL import MyIBL as IBL
from sklearn.linear_model import LogisticRegression as LG

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [4]:
def getData(path, filenames_type):
    '''
    features_lst = [
        "Pclass", "Survived", "Name", "Sex", "Age",
        "Sibsp", "Parch", "Ticket", "Fare", "Cabin","Embarked"]
    '''
    if filenames_type == 'train':
        filename = 'train'
    elif filenames_type == 'test':
        filename = 'test'
    else:
        filename = 'titanicAll'
    df_features = pd.read_csv(path + filename + '.csv',
                           sep=',')

    if filenames_type not in ['train', 'test']:
        # drop unnecessary columns that don't exist in the official dataset
        df_features.drop(['Boat', 'Body', 'Home.dest'],
                          axis=1,
                         inplace=True)
    #labels = df_features['Survived']
    #df_features = df_features.drop(['Survived'], axis=1)
    return df_features

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "-c", "--config", default="titanic.cfg",
    help="specify the location of the clustering config file"
)
args, _ = parser.parse_known_args()

config_file = args.config
config = load(config_file)

##
verbose = config.get('titanic', 'verbose')
path = config.get('titanic', 'path') + '/'
file_type = config.get('titanic', 'file_type')

filename_type = 'train'
if file_type == 'all':
    filename_type = 'other'

In [6]:
print('Filename type:', filename_type)
print()
## train
trainData = getData(path, filename_type)
# Preprocessing
trainPreprocess = MyPreprocessing(process_type='all',
                                  filename_type=filename_type)

## test
filename_type = 'test'
testData = getData(path, filename_type)
# Preprocessing
testPreprocess = MyPreprocessing(process_type='all',
                                 filename_type=filename_type,
                                 remove_outliers=False)

Filename type: train



In [7]:
trainPreprocess.fit(trainData)
df_train = trainPreprocess.new_df
# the labels "Survived"
labels = trainPreprocess.labels_
#print(labels.head())
# the initial dataset without any preprocessing
#print(trainPreprocess.df_initial.head())
# the preprocessed data
#print(trainPreprocess.new_df.head())

testPreprocess.fit(testData)
df_test = testPreprocess.new_df

process_type: all
Number of outliers 25
process_type: all


In [8]:
# fix missing columns because of NaNs and one hot encoding without dummy_na
if df_train.shape[1] != df_test.shape[1]:
    missing_cols = set(df_test.columns) - set(df_train.columns)
    for col in missing_cols:
        #df_train[col] = np.zeros([df_train.shape[0], 1])
        df_test.drop([col], axis=1, inplace=True)

    missing_cols = set(df_train.columns) - set(df_test.columns)
    for col in missing_cols:
        #df_test[col] = np.zeros([df_test.shape[0], 1])
        df_train.drop([col], axis=1, inplace=True)

labels_test = testPreprocess.labels_

print(df_train.columns, df_test.columns)
print(df_train.shape, df_test.shape)

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'PclassCp_2', 'PclassCp_3', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'FamilySize', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'Family_bin_SmallFamily', 'Family_bin_BigFamily', 'Family_bin_Team', 'Fare_bin_Median', 'Fare_bin_Average', 'Fare_bin_High', 'Mother', 'Father', 'Daughter', 'Son', 'Orphan', 'RichWoman', 'MiddleClassWoman', 'PoorWoman', 'RichMan', 'MiddleClassMan', 'PoorMan', 'RichGirl', 'MiddleClassGirl', 'PoorGirl', 'RichBoy', 'MiddleClassBoy', 'PoorBoy'], dtype='object') Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'PclassCp_2', 'PclassCp_3', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'FamilySize', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'Family_bin_SmallFamily', 'Fam

In [9]:
df_train.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'PclassCp_2', 'PclassCp_3', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'FamilySize', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'Family_bin_SmallFamily', 'Family_bin_BigFamily', 'Family_bin_Team', 'Fare_bin_Median', 'Fare_bin_Average', 'Fare_bin_High', 'Mother', 'Father', 'Daughter', 'Son', 'Orphan', 'RichWoman', 'MiddleClassWoman', 'PoorWoman', 'RichMan', 'MiddleClassMan', 'PoorMan', 'RichGirl', 'MiddleClassGirl', 'PoorGirl', 'RichBoy', 'MiddleClassBoy', 'PoorBoy'], dtype='object')

In [10]:
df_test.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'PclassCp_2', 'PclassCp_3', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'FamilySize', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'Family_bin_SmallFamily', 'Family_bin_BigFamily', 'Family_bin_Team', 'Fare_bin_Median', 'Fare_bin_Average', 'Fare_bin_High', 'Mother', 'Father', 'Daughter', 'Son', 'Orphan', 'RichWoman', 'MiddleClassWoman', 'PoorWoman', 'RichMan', 'MiddleClassMan', 'PoorMan', 'RichGirl', 'MiddleClassGirl', 'PoorGirl', 'RichBoy', 'MiddleClassBoy', 'PoorBoy'], dtype='object')

In [11]:
#df_test.dtypes

In [12]:
def feature_importance(clf, df_train):
    if hasattr(clf, 'feature_importances_'):
        name = str(clf.__class__).split('.')[-1][:-2]
        feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
        feat_imp['feature'] = df_train.columns
        feat_imp.sort_values(by='importance', ascending=False, inplace=True)
        #feat_imp = feat_imp.iloc[:top_n]
        feat_imp.sort_values(by='importance', inplace=True)
        feat_imp = feat_imp.set_index('feature', drop=True)
        feat_imp.plot.barh(title='Feature Importance', figsize=(10,10))
        plt.xlabel('%s Feature Importance Score' %name)
        plt.show()
        return feat_imp
    return pd.DataFrame()

In [13]:
def ensemble(clf, x_train, y_train, x_test):
    
    name = str(clf.__class__).split('.')[-1][:-2]
    kf = KFold(n_splits=5)

    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]

    #ens_train = np.zeros((ntrain,))
    ens_train = np.array([])
    #ens_test = np.zeros((ntest,))
    #test_kf = np.empty((kf.n_folds, ntest))
    test_kf = np.zeros((kf.n_splits, ntest))
        
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.fit(x_tr, y_tr)

        #ens_train[test_index] = clf.predict(x_te)
        ens_train = np.concatenate([ens_train, clf.predict(x_te)])
        # predict from the whole train set 
        #clf.fit(x_train,y_train)
        test_kf[i, :] = clf.predict(x_test)
    
    # features for next ensemble layer
    ens_train = pd.DataFrame(ens_train.reshape(-1,1), columns=[name])
    
    # combine for test
    ens_test = pd.DataFrame(test_kf.T).mode(axis=1)
    ens_test.columns = [name]
    
    return ens_train, ens_test

In [14]:
def modeler(clf, data, labels, train_fidx, validation_fidx, name_prefix):
    name = name_prefix + str(clf.__class__).split('.')[-1][:-2]
    #print(name)
    folds_accuracy = list()
    duration = list()
    start = time()
    for idx, trf in enumerate(train_fidx):
        clf.fit(data.loc[trf], labels.loc[trf])
        prediction_labels = clf.predict(data.loc[validation_fidx[idx]])
        folds_accuracy.append(accuracy_score(labels.loc[validation_fidx[idx]], prediction_labels))
        
    mean_acc = mean(folds_accuracy)
    duration = time() - start
    #print(name, mean_acc, "accuracy at validation stage,", duration, 's')
    df_folds_accuracy = pd.DataFrame([folds_accuracy], index=[name])
    df_folds = pd.DataFrame([[mean_acc, duration/len(train_fidx)]], columns=['Accuracy', 'Duration'], index=[name])
    return clf.fit(data, labels), df_folds, df_folds_accuracy

In [15]:
def get_cv_data(df_train, cv=5):
    kf = KFold(n_splits=cv)
    folds = [(train_idx, validation_idx) for train_idx, validation_idx in kf.split(df_train)]
    train_idx = [f[0] for f in folds]
    validation_idx = [f[1] for f in folds]
    return train_idx, validation_idx

In [16]:
def run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True):
    models = []
    df_ens_features = pd.DataFrame()
    df_ens_test = pd.DataFrame()
    df_model_cvaccuracies = pd.DataFrame()
    df_model_info = pd.DataFrame()
    for clf in clfs:
        name = str(clf.__class__).split('.')[-1][:-2]
        
        # cross validation
        if cv:
            model, df_folds, df_folds_accuracy = modeler(clf, df_train, labels, train_idx, validation_idx, name_prefix)
            models.append(model)
            df_model_cvaccuracies = pd.concat([df_model_cvaccuracies, df_folds_accuracy], sort=False)
            df_model_info = pd.concat([df_model_info, df_folds], sort=False)  
            
            #feature_importance(clf, df_train)
        
        # building ensemble
        if cv and ens:
            #df_feature = pd.DataFrame(cross_val_predict(clf, df_train, labels, cv=ens_kf).reshape(-1,1), columns=[name])
            df_ens_feature, ens_test = ensemble(clf, df_train, labels, df_test)
            df_ens_features = pd.concat([df_ens_features, df_ens_feature], axis=1, sort=False).reset_index(drop=True)
            df_ens_test = pd.concat([df_ens_test, ens_test], axis=1, sort=False).reset_index(drop=True)
            
    return models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test
        

In [17]:
clfs = [IBL(), RF(n_estimators =12), SVC(gamma='scale'), XGB(), MLP(max_iter=1000), KNN(), LDA()]
#clfs = [RF(n_estimators =12), SVC(gamma='scale'), XGB(), KNN(), LDA()]

In [18]:
train_idx, validation_idx = get_cv_data(df_train, cv=5)

In [19]:
import warnings
warnings.filterwarnings("ignore")

In [52]:
print('Original')
print('##################################')
name_prefix = 'ALL_' + str(len(df_train.columns)) + '_'
models, df_results_all_cv, df_results_all, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
df_results_all

Original
##################################


,Accuracy,Duration
ALL_49_MyIBL,0.750575,2.490479
ALL_49_RandomForestClassifier,0.807142,0.026679
ALL_49_SVC,0.821009,0.033901
ALL_49_XGBClassifier,0.822198,0.140709
ALL_49_MLPClassifier,0.799103,4.260149
ALL_49_KNeighborsClassifier,0.807196,0.015391
ALL_49_LinearDiscriminantAnalysis,0.814099,0.018390


In [22]:
print('PCA')
print('##################################')
df_results_pca = pd.DataFrame()
df_results_pca_cv = pd.DataFrame()
for n_dim in range(15, 22):
    print(n_dim, ' dimensions')
    pca_train, pca_test, ev = MyFeatureSelection.applyPCA(df_train, df_test, n_dim)
    name_prefix = 'PCA_d_' + str(n_dim) + '/' + str(len(df_train.columns)) + '_ev_' + str(round(ev,3)) + '_'
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_pca_cv = pd.concat([df_results_pca_cv, df_model_cvaccuracies], axis=0)
    df_results_pca = pd.concat([df_results_pca, df_model_info], axis=0)

PCA
##################################
15  dimensions
16  dimensions
17  dimensions
18  dimensions
19  dimensions
20  dimensions
21  dimensions


In [23]:
df_results_pca

,Accuracy,Duration
PCA_d_15/49_ev_0.871_MyIBL,0.720563,2.548038
PCA_d_15/49_ev_0.871_RandomForestClassifier,0.811807,0.020154
PCA_d_15/49_ev_0.871_SVC,0.821009,0.031275
PCA_d_15/49_ev_0.871_XGBClassifier,0.822198,0.131004
PCA_d_15/49_ev_0.871_MLPClassifier,0.797934,4.218026
PCA_d_15/49_ev_0.871_KNeighborsClassifier,0.807196,0.013992
PCA_d_15/49_ev_0.871_LinearDiscriminantAnalysis,0.814099,0.018456
PCA_d_16/49_ev_0.885_MyIBL,0.778334,2.536446
PCA_d_16/49_ev_0.885_RandomForestClassifier,0.807182,0.022253
PCA_d_16/49_ev_0.885_SVC,0.821009,0.032779


In [24]:
df_results_pca_order = df_results_pca.copy()
df_results_pca_order.sort_values('Accuracy')

,Accuracy,Duration
PCA_d_15/49_ev_0.871_MyIBL,0.720563,2.548038
PCA_d_20/49_ev_0.932_MyIBL,0.743718,2.394284
PCA_d_19/49_ev_0.922_MyIBL,0.744854,2.431328
PCA_d_17/49_ev_0.898_MyIBL,0.755199,2.472280
PCA_d_21/49_ev_0.941_MyIBL,0.758720,2.444081
PCA_d_18/49_ev_0.91_MyIBL,0.763278,2.357697
PCA_d_16/49_ev_0.885_MyIBL,0.778334,2.536446
PCA_d_15/49_ev_0.871_MLPClassifier,0.797934,4.218026
PCA_d_18/49_ev_0.91_RandomForestClassifier,0.799063,0.029182
PCA_d_21/49_ev_0.941_RandomForestClassifier,0.799090,0.021874


In [25]:
df_results_pca_cv

,0,1,2,3,4
PCA_d_15/49_ev_0.871_MyIBL,0.712644,0.722543,0.693642,0.763006,0.710983
PCA_d_15/49_ev_0.871_RandomForestClassifier,0.787356,0.797688,0.826590,0.809249,0.838150
PCA_d_15/49_ev_0.871_SVC,0.827586,0.809249,0.815029,0.797688,0.855491
PCA_d_15/49_ev_0.871_XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
PCA_d_15/49_ev_0.871_MLPClassifier,0.787356,0.780347,0.774566,0.809249,0.838150
PCA_d_15/49_ev_0.871_KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
PCA_d_15/49_ev_0.871_LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
PCA_d_16/49_ev_0.885_MyIBL,0.741379,0.745665,0.768786,0.809249,0.826590
PCA_d_16/49_ev_0.885_RandomForestClassifier,0.787356,0.803468,0.826590,0.780347,0.838150
PCA_d_16/49_ev_0.885_SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [26]:
max(df_results_pca['Accuracy'])

0.8221978606072686

In [27]:
print('ICA')
print('##################################')
df_results_ica = pd.DataFrame()
df_results_ica_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    ica_train, ica_test = MyFeatureSelection.applyICA(df_train, df_test, n_dim)
    name_prefix = 'ICA_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_ica_cv = pd.concat([df_results_ica_cv, df_model_cvaccuracies], axis=0)
    df_results_ica = pd.concat([df_results_ica, df_model_info], axis=0)

ICA
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [28]:
df_results_ica

,Accuracy,Duration
ICA_d_25/49MyIBL,0.755199,2.395105
ICA_d_25/49RandomForestClassifier,0.799083,0.024999
ICA_d_25/49SVC,0.821009,0.028124
ICA_d_25/49XGBClassifier,0.822198,0.124996
ICA_d_25/49MLPClassifier,0.799083,4.099877
ICA_d_25/49KNeighborsClassifier,0.807196,0.012500
ICA_d_25/49LinearDiscriminantAnalysis,0.814099,0.015625
ICA_d_30/49MyIBL,0.747126,2.423865
ICA_d_30/49RandomForestClassifier,0.801389,0.021875
ICA_d_30/49SVC,0.821009,0.031249


In [29]:
df_results_ica_cv

,0,1,2,3,4
ICA_d_25/49MyIBL,0.752874,0.728324,0.751445,0.797688,0.745665
ICA_d_25/49RandomForestClassifier,0.793103,0.786127,0.820809,0.791908,0.803468
ICA_d_25/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
ICA_d_25/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
ICA_d_25/49MLPClassifier,0.793103,0.780347,0.797688,0.786127,0.838150
ICA_d_25/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
ICA_d_25/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
ICA_d_30/49MyIBL,0.735632,0.687861,0.780347,0.768786,0.763006
ICA_d_30/49RandomForestClassifier,0.798851,0.786127,0.820809,0.780347,0.820809
ICA_d_30/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [30]:
df_results_ica_order = df_results_ica.copy()
df_results_ica_order.sort_values('Accuracy')

,Accuracy,Duration
ICA_d_45/49MyIBL,0.736808,2.340555
ICA_d_40/49MyIBL,0.743678,2.295515
ICA_d_30/49MyIBL,0.747126,2.423865
ICA_d_35/49MyIBL,0.754063,2.422904
ICA_d_25/49MyIBL,0.755199,2.395105
ICA_d_30/49MLPClassifier,0.792167,4.053257
ICA_d_25/49RandomForestClassifier,0.799083,0.024999
ICA_d_25/49MLPClassifier,0.799083,4.099877
ICA_d_45/49MLPClassifier,0.799103,4.387111
ICA_d_40/49RandomForestClassifier,0.800219,0.021874


In [31]:
max(df_results_ica['Accuracy'])

0.8221978606072686

In [32]:
print('INFO GAIN SELECTION')
print('##################################')
df_results_ig = pd.DataFrame()
df_results_ig_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    ig_train, ig_test = MyFeatureSelection.InfoGainSelection(df_train, df_test, labels, n_dim)
    name_prefix = 'IG_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_ig_cv = pd.concat([df_results_ig_cv, df_model_cvaccuracies], axis=0)
    df_results_ig = pd.concat([df_results_ig, df_model_info], axis=0)

INFO GAIN SELECTION
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [33]:
df_results_ig

,Accuracy,Duration
IG_d_25/49MyIBL,0.732171,2.317642
IG_d_25/49RandomForestClassifier,0.811807,0.021874
IG_d_25/49SVC,0.821009,0.028124
IG_d_25/49XGBClassifier,0.822198,0.128121
IG_d_25/49MLPClassifier,0.806046,4.184964
IG_d_25/49KNeighborsClassifier,0.807196,0.012500
IG_d_25/49LinearDiscriminantAnalysis,0.814099,0.018749
IG_d_30/49MyIBL,0.732137,2.371804
IG_d_30/49RandomForestClassifier,0.803727,0.021874
IG_d_30/49SVC,0.821009,0.031249


In [34]:
df_results_ig_order = df_results_ig.copy()
df_results_ig_order.sort_values('Accuracy')

,Accuracy,Duration
IG_d_45/49MyIBL,0.730968,2.540549
IG_d_30/49MyIBL,0.732137,2.371804
IG_d_25/49MyIBL,0.732171,2.317642
IG_d_40/49MyIBL,0.749399,2.515550
IG_d_35/49MyIBL,0.757584,2.509300
IG_d_35/49MLPClassifier,0.800252,3.824886
IG_d_35/49RandomForestClassifier,0.801409,0.024999
IG_d_45/49MLPClassifier,0.802565,3.368649
IG_d_30/49MLPClassifier,0.802565,4.081128
IG_d_30/49RandomForestClassifier,0.803727,0.021874


In [35]:
df_results_ig_cv

,0,1,2,3,4
IG_d_25/49MyIBL,0.672414,0.699422,0.751445,0.774566,0.763006
IG_d_25/49RandomForestClassifier,0.787356,0.791908,0.832370,0.786127,0.861272
IG_d_25/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
IG_d_25/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
IG_d_25/49MLPClassifier,0.770115,0.786127,0.826590,0.791908,0.855491
IG_d_25/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
IG_d_25/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
IG_d_30/49MyIBL,0.701149,0.716763,0.728324,0.768786,0.745665
IG_d_30/49RandomForestClassifier,0.775862,0.786127,0.838150,0.809249,0.809249
IG_d_30/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [36]:
max(df_results_ig['Accuracy'])

0.8221978606072686

In [37]:
print('ANOVA SELECTION')
print('##################################')
df_results_an = pd.DataFrame()
df_results_an_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    an_train, an_test = MyFeatureSelection.AnovaSelection(df_train, df_test, labels, n_dim)
    name_prefix = 'AN_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_an_cv = pd.concat([df_results_an_cv, df_model_cvaccuracies], axis=0)
    df_results_an = pd.concat([df_results_an, df_model_info], axis=0)

ANOVA SELECTION
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [38]:
df_results_an

,Accuracy,Duration
AN_d_25/49MyIBL,0.762142,2.511650
AN_d_25/49RandomForestClassifier,0.797927,0.024999
AN_d_25/49SVC,0.821009,0.031249
AN_d_25/49XGBClassifier,0.822198,0.139813
AN_d_25/49MLPClassifier,0.797934,4.709234
AN_d_25/49KNeighborsClassifier,0.807196,0.012500
AN_d_25/49LinearDiscriminantAnalysis,0.814099,0.015623
AN_d_30/49MyIBL,0.712531,2.453633
AN_d_30/49RandomForestClassifier,0.808332,0.021874
AN_d_30/49SVC,0.821009,0.031249


In [39]:
df_results_an_order = df_results_an.copy()
df_results_an_order.sort_values('Accuracy')

,Accuracy,Duration
AN_d_30/49MyIBL,0.712531,2.453633
AN_d_40/49MyIBL,0.733267,2.344344
AN_d_45/49MyIBL,0.750608,2.387445
AN_d_35/49MyIBL,0.762122,2.442493
AN_d_25/49MyIBL,0.762142,2.511650
AN_d_40/49RandomForestClassifier,0.795641,0.021874
AN_d_30/49MLPClassifier,0.796798,4.618069
AN_d_25/49RandomForestClassifier,0.797927,0.024999
AN_d_25/49MLPClassifier,0.797934,4.709234
AN_d_40/49MLPClassifier,0.799083,4.156125


In [40]:
df_results_an_cv

,0,1,2,3,4
AN_d_25/49MyIBL,0.747126,0.722543,0.757225,0.809249,0.774566
AN_d_25/49RandomForestClassifier,0.793103,0.774566,0.809249,0.780347,0.832370
AN_d_25/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
AN_d_25/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
AN_d_25/49MLPClassifier,0.787356,0.780347,0.786127,0.797688,0.838150
AN_d_25/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
AN_d_25/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
AN_d_30/49MyIBL,0.660920,0.693642,0.722543,0.739884,0.745665
AN_d_30/49RandomForestClassifier,0.793103,0.780347,0.838150,0.786127,0.843931
AN_d_30/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [41]:
max(df_results_an['Accuracy'])

0.8221978606072686

In [42]:
# Random forest estimator for feature selection
print('Random Forest Estimator Selection')
print('##################################')
rf_train, rf_test = MyFeatureSelection.RandomForestSelection(df_train, df_test, labels, 100)
print(len(rf_train.columns), ' dimensions')
name_prefix = 'RF_d_' + str(len(rf_train.columns)) + '/' + str(len(df_train.columns))
models, df_results_rf_cv, df_results_rf, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)



Random Forest Estimator Selection
##################################
25  dimensions


In [43]:
df_results_rf

,Accuracy,Duration
RF_d_25/49MyIBL,0.722889,2.376804
RF_d_25/49RandomForestClassifier,0.800233,0.021874
RF_d_25/49SVC,0.821009,0.031249
RF_d_25/49XGBClassifier,0.822198,0.128121
RF_d_25/49MLPClassifier,0.802565,3.876571
RF_d_25/49KNeighborsClassifier,0.807196,0.012500
RF_d_25/49LinearDiscriminantAnalysis,0.814099,0.018749


In [44]:
df_results_rf_cv

,0,1,2,3,4
RF_d_25/49MyIBL,0.701149,0.687861,0.728324,0.815029,0.682081
RF_d_25/49RandomForestClassifier,0.798851,0.797688,0.797688,0.768786,0.838150
RF_d_25/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
RF_d_25/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
RF_d_25/49MLPClassifier,0.781609,0.780347,0.826590,0.791908,0.832370
RF_d_25/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
RF_d_25/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711


In [45]:
print('Lasso Regression Selection')
print('##################################')
df_results_lr = pd.DataFrame()
df_results_lr_cv = pd.DataFrame()
for alpha in [0.003, 0.0009, 0.0005, 0.0002, 0.00001]:
    lr_train, lr_test = MyFeatureSelection.LassoRegressionSelection(df_train, df_test, labels, alpha)
    print(len(lr_train.columns), ' dimensions')
    name_prefix = 'LR_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_lr_cv = pd.concat([df_results_lr_cv, df_model_cvaccuracies], axis=0)
    df_results_lr = pd.concat([df_results_lr, df_model_info], axis=0)

Lasso Regression Selection
##################################
18  dimensions
29  dimensions
37  dimensions
40  dimensions
45  dimensions


In [46]:
df_results_lr

,Accuracy,Duration
LR_d_45/49MyIBL,0.767916,2.436862
LR_d_45/49RandomForestClassifier,0.807176,0.021874
LR_d_45/49SVC,0.821009,0.028124
LR_d_45/49XGBClassifier,0.822198,0.124996
LR_d_45/49MLPClassifier,0.799096,4.156125
LR_d_45/49KNeighborsClassifier,0.807196,0.015625
LR_d_45/49LinearDiscriminantAnalysis,0.814099,0.015625
LR_d_45/49MyIBL,0.747173,2.352246
LR_d_45/49RandomForestClassifier,0.800239,0.021875
LR_d_45/49SVC,0.821009,0.031249


In [47]:
df_results_lr_cv

,0,1,2,3,4
LR_d_45/49MyIBL,0.752874,0.739884,0.815029,0.774566,0.757225
LR_d_45/49RandomForestClassifier,0.793103,0.797688,0.809249,0.791908,0.843931
LR_d_45/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491
LR_d_45/49XGBClassifier,0.798851,0.815029,0.832370,0.797688,0.867052
LR_d_45/49MLPClassifier,0.781609,0.780347,0.809249,0.791908,0.832370
LR_d_45/49KNeighborsClassifier,0.775862,0.803468,0.820809,0.809249,0.826590
LR_d_45/49LinearDiscriminantAnalysis,0.804598,0.815029,0.809249,0.791908,0.849711
LR_d_45/49MyIBL,0.695402,0.693642,0.774566,0.797688,0.774566
LR_d_45/49RandomForestClassifier,0.793103,0.780347,0.809249,0.803468,0.815029
LR_d_45/49SVC,0.827586,0.809249,0.815029,0.797688,0.855491


In [48]:
df_results_lr_order = df_results_lr.copy()
df_results_lr_order.sort_values('Accuracy')

,Accuracy,Duration
LR_d_45/49MyIBL,0.746004,2.330366
LR_d_45/49MyIBL,0.747153,2.337786
LR_d_45/49MyIBL,0.747173,2.352246
LR_d_45/49MyIBL,0.757511,2.383792
LR_d_45/49MyIBL,0.767916,2.436862
LR_d_45/49MLPClassifier,0.796798,4.371744
LR_d_45/49MLPClassifier,0.799096,4.156125
LR_d_45/49RandomForestClassifier,0.800239,0.021875
LR_d_45/49MLPClassifier,0.802565,3.928008
LR_d_45/49MLPClassifier,0.804863,4.128002


In [53]:
df_total_order =  pd.concat([df_results_all, df_results_pca, df_results_ica, df_results_ig, df_results_an, df_results_rf, df_results_lr], axis=0)
df_total_order.sort_values('Accuracy')

,Accuracy,Duration
AN_d_30/49MyIBL,0.712531,2.453633
PCA_d_15/49_ev_0.871_MyIBL,0.720563,2.548038
RF_d_25/49MyIBL,0.722889,2.376804
IG_d_45/49MyIBL,0.730968,2.540549
IG_d_30/49MyIBL,0.732137,2.371804
IG_d_25/49MyIBL,0.732171,2.317642
AN_d_40/49MyIBL,0.733267,2.344344
ICA_d_45/49MyIBL,0.736808,2.340555
ICA_d_40/49MyIBL,0.743678,2.295515
PCA_d_20/49_ev_0.932_MyIBL,0.743718,2.394284


In [54]:
df_results_all

,Accuracy,Duration
ALL_49_MyIBL,0.750575,2.490479
ALL_49_RandomForestClassifier,0.807142,0.026679
ALL_49_SVC,0.821009,0.033901
ALL_49_XGBClassifier,0.822198,0.140709
ALL_49_MLPClassifier,0.799103,4.260149
ALL_49_KNeighborsClassifier,0.807196,0.015391
ALL_49_LinearDiscriminantAnalysis,0.814099,0.018390


In [58]:
lr_train.shape

(866, 45)